In [90]:
import mysql.connector as sql
import pandas as pd
import datetime
import uuid
import numpy as np
from itertools import combinations

In [91]:
db_connection = sql.connect(host='localhost', database='benedict', user='root', password='artema2')

In [110]:
# Select 50 patients for the test
qry = "SELECT DISTINCT fin as patient_id FROM dka_case_events "
qry = qry + " ORDER BY patient_id LIMIT 50 "
print(qry)
pat_df = pd.read_sql(qry, con=db_connection)

# Convert patient_id Series (column) to a list of values
pat_list = list(pat_df['patient_id'])
print(pat_list)

SELECT DISTINCT fin as patient_id FROM dka_case_events  ORDER BY patient_id LIMIT 50 
['++Xqt3mDJtPJ', '+/xuDGznPP1X', '+0KNKRJdrb2U', '+16f+oeBrbDV', '+1HeBTXSNHLh', '+2hf8/i2b1dN', '+2Tdh1HPB1J9', '+3SLlT5NFXJY', '+4brKZG8kc46', '+4gPKkqA+1Ia', '+69fVhKM8Cry', '+7faW7ZDI4fZ', '+8n6M0NZIhEi', '+9UE8d8sSh1o', '+A+ScGXz5LWx', '+aH6x0VwbiHa', '+bjhPAUDOeYs', '+d6D5wOLSMYv', '+eDuUtS4WjYZ', '+EZ6d3eyQWyI', '+F8+ZIVSdPM2', '+FANhCqSz6uL', '+G+9FYulca1t', '+GrAH6YG7lu1', '+Hjq4vw0gNl2', '+HP+Ve9FIK68', '+Hu2bx3waSb4', '+ICzBG17UkcU', '+IDJ5oh/qitO', '+jLAcGqK0co6', '+jqcvLIhNXiN', '+JXUcPso3SGp', '+kpo08j/PvRQ', '+kV119+gi1I8', '+l1AXCRGz9dL', '+lEn4iiEVTmX', '+NbUKF5T2ZWv', '+Oa5moEd94KC', '+OUaIf6RZa0f', '+p7UkwhBogV6', '+PD8EsRCoGuc', '+POtoM1xxGTY', '+qG1jEsgtTqo', '+Qi+1pIgC1vV', '+RABr/FaHCps', '+RmcqjlwMzdh', '+s3JtdC1ehNf', '+SSnDJnFgGGP', '+syDkxZWo4SJ', '+t3UFcERP4ju']


In [111]:
qry = "SELECT auto_id, fin as patient_id, event_date, event_name, result_val, result_unit, event_category "
qry = qry + " FROM dka_case_events "
qry = qry + " WHERE fin IN (" + str(pat_list).replace('[', '').replace(']', '') + ")"
qry = qry + " ORDER BY patient_id, event_date "


print(qry)

df = pd.read_sql(qry, con=db_connection)


SELECT auto_id, fin as patient_id, event_date, event_name, result_val, result_unit, event_category  FROM dka_case_events  WHERE fin IN ('++Xqt3mDJtPJ', '+/xuDGznPP1X', '+0KNKRJdrb2U', '+16f+oeBrbDV', '+1HeBTXSNHLh', '+2hf8/i2b1dN', '+2Tdh1HPB1J9', '+3SLlT5NFXJY', '+4brKZG8kc46', '+4gPKkqA+1Ia', '+69fVhKM8Cry', '+7faW7ZDI4fZ', '+8n6M0NZIhEi', '+9UE8d8sSh1o', '+A+ScGXz5LWx', '+aH6x0VwbiHa', '+bjhPAUDOeYs', '+d6D5wOLSMYv', '+eDuUtS4WjYZ', '+EZ6d3eyQWyI', '+F8+ZIVSdPM2', '+FANhCqSz6uL', '+G+9FYulca1t', '+GrAH6YG7lu1', '+Hjq4vw0gNl2', '+HP+Ve9FIK68', '+Hu2bx3waSb4', '+ICzBG17UkcU', '+IDJ5oh/qitO', '+jLAcGqK0co6', '+jqcvLIhNXiN', '+JXUcPso3SGp', '+kpo08j/PvRQ', '+kV119+gi1I8', '+l1AXCRGz9dL', '+lEn4iiEVTmX', '+NbUKF5T2ZWv', '+Oa5moEd94KC', '+OUaIf6RZa0f', '+p7UkwhBogV6', '+PD8EsRCoGuc', '+POtoM1xxGTY', '+qG1jEsgtTqo', '+Qi+1pIgC1vV', '+RABr/FaHCps', '+RmcqjlwMzdh', '+s3JtdC1ehNf', '+SSnDJnFgGGP', '+syDkxZWo4SJ', '+t3UFcERP4ju') ORDER BY patient_id, event_date 


In [112]:
df.shape

(65038, 7)

In [190]:
df.to_csv('dka_events.csv')
df.head()

,auto_id,patient_id,event_date,event_name,result_val,result_unit,event_category
0,2665960,++Xqt3mDJtPJ,2014-01-02 10:00:00,Systolic BP,141.0,mmHg,vitals
1,2136738,++Xqt3mDJtPJ,2014-01-02 10:00:00,Respiratory Rate,18.0,RPM,vitals
2,1522574,++Xqt3mDJtPJ,2014-01-02 10:00:00,Pulse,68.0,BPM,vitals
3,448571,++Xqt3mDJtPJ,2014-01-02 10:00:00,Diastolic BP,61.0,mmHg,vitals
4,4545,++Xqt3mDJtPJ,2014-01-02 10:00:00,Diastolic BP,61.0,mmHg,vitals


* Get a list of unique patients
    * For each patient, get a list of events, sorted by timestamp
        * For each event ID, find all events that occur at the same time and create a "COOCCUR" edge
        * Once all co-occurances have been identified, find all sequential events and CREATE "BEFORE" edge

## Figure out co-occurances

In [114]:
grouped_df = df.groupby(['patient_id','event_date']).size().reset_index()
grouped_df.head()

,patient_id,event_date,0
0,++Xqt3mDJtPJ,2014-01-02 10:00:00,6
1,++Xqt3mDJtPJ,2014-01-02 10:05:00,7
2,++Xqt3mDJtPJ,2014-01-02 13:10:00,1
3,++Xqt3mDJtPJ,2014-01-02 13:14:00,3
4,++Xqt3mDJtPJ,2014-01-02 13:15:00,5


In [115]:
grouped_df.shape

(16841, 3)

In [117]:
cooccurances = []
for idx, row in grouped_df.iterrows():
    q = 'patient_id == "' + row['patient_id'] + '" and event_date == "' + str(row['event_date']) + '"'
    #print(q)
    
    cooccur_df = df.query(q)
    if cooccur_df.shape[0] > 0:
        
        events = list(cooccur_df['auto_id'])
        #print(events)
        comb = combinations(events, 2) 
        cooccurances = cooccurances + list(comb)



In [176]:
# Drop duplicates
cooccurances = list(set(cooccurances))
cooccur_df = pd.DataFrame(cooccurances, columns = ['event_id','co_occur_event_id'])
cooccur_df.head()

,event_id,co_occur_event_id
0,3030274,2570310
1,3213420,3266299
2,487541,949275
3,2503132,1323741
4,1333121,320282


In [177]:
cooccur_df.to_csv('cooccurances.csv')

In [178]:
cooccur_df.shape

(129969, 2)

## Temporal sequences

In [167]:
df.head()

,auto_id,patient_id,event_date,event_name,result_val,result_unit,event_category
0,2665960,++Xqt3mDJtPJ,2014-01-02 10:00:00,Systolic BP,141.0,mmHg,vitals
1,2136738,++Xqt3mDJtPJ,2014-01-02 10:00:00,Respiratory Rate,18.0,RPM,vitals
2,1522574,++Xqt3mDJtPJ,2014-01-02 10:00:00,Pulse,68.0,BPM,vitals
3,448571,++Xqt3mDJtPJ,2014-01-02 10:00:00,Diastolic BP,61.0,mmHg,vitals
4,4545,++Xqt3mDJtPJ,2014-01-02 10:00:00,Diastolic BP,61.0,mmHg,vitals


In [174]:
seq = []
for idx, row in df.iterrows():
    #print(row['auto_id'])
    q = 'patient_id == "' + row['patient_id'] + '"'
    q = q + ' and event_date > "' + str(row['event_date']) + '"'
    #print(q)
    next_events = df.query(q).sort_values('event_date')
    if next_events.shape[0] > 0:
        next_event_id = df.query(q).sort_values('event_date').iloc[0]['auto_id']
        temp = [row['auto_id'], next_event_id]
        seq.append(temp)
    #break
print(seq)

[[2665960, 3106599], [2136738, 3106599], [1522574, 3106599], [448571, 3106599], [4545, 3106599], [894763, 3106599], [3106599, 1522575], [3159400, 1522575], [3213407, 1522575], [3266286, 1522575], [3320696, 1522575], [3373328, 1522575], [3424574, 1522575], [1522575, 448572], [448572, 2665962], [4546, 2665962], [2665961, 2665962], [2665962, 448574], [4547, 448574], [1522576, 448574], [894764, 448574], [448573, 448574], [448574, 1522577], [4548, 1522577], [2665963, 1522577], [1522577, 4549], [894765, 4549], [4549, 2665965], [448575, 2665965], [2665964, 2665965], [2665965, 4551], [448576, 4551], [4550, 4551], [4551, 2665967], [448577, 2665967], [894766, 2665967], [1522578, 2665967], [2665966, 2665967], [2665967, 4553], [448578, 4553], [4552, 4553], [4553, 2665969], [448579, 2665969], [1522579, 2665969], [894767, 2665969], [2665968, 2665969], [2665969, 4555], [448580, 4555], [4554, 4555], [4555, 2665971], [448581, 2665971], [2665970, 2665971], [2665971, 1522581], [448582, 1522581], [4556, 1

In [179]:
seq_df = pd.DataFrame(seq, columns=['event_id_before','event_id_after'])
seq_df.to_csv('sequences.csv')

In [180]:
seq_df.shape

(64874, 2)